<a href="https://colab.research.google.com/github/ssobranding/Portfolio/blob/main/3_DM_G2_Luxury_E_Commerce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Creating the  Database

import sqlite3

# Connect to SQLite (creates 'ecommerce.db' file)
db_filename = "luxury_ecommerce.db"
conn = sqlite3.connect(db_filename)
cursor = conn.cursor()

# Create tables
schema_script = """

-- User Table
CREATE TABLE IF NOT EXISTS User (
    UserID INTEGER PRIMARY KEY AUTOINCREMENT,
    Email TEXT UNIQUE NOT NULL,
    Password TEXT NOT NULL,
    FirstName TEXT,
    LastName TEXT,
    MemberType TEXT CHECK(MemberType IN ('Silver', 'Gold', 'Diamond')) NOT NULL,
    PhoneNumber TEXT
);

-- CardInfo Table
CREATE TABLE IF NOT EXISTS CardInfo (
    CardID INTEGER PRIMARY KEY AUTOINCREMENT,
    UserID INTEGER NOT NULL,
    CardNumber TEXT NOT NULL,
    CVV TEXT NOT NULL,
    ExpiryDate TEXT NOT NULL,
    FOREIGN KEY (UserID) REFERENCES User(UserID) ON DELETE CASCADE
);

-- ContactDetail Table
CREATE TABLE IF NOT EXISTS ContactDetail (
    AddressID INTEGER PRIMARY KEY AUTOINCREMENT,
    UserID INTEGER NOT NULL,
    Country TEXT,
    State TEXT,
    City TEXT,
    Street1 TEXT,
    Street2 TEXT,
    Address TEXT,
    FOREIGN KEY (UserID) REFERENCES User(UserID) ON DELETE CASCADE
);

-- OrderDetail Table
CREATE TABLE IF NOT EXISTS OrderDetail (
    OrderID INTEGER PRIMARY KEY AUTOINCREMENT,
    UserID INTEGER NOT NULL,
    OrderDate TEXT NOT NULL,
    OrderStatus TEXT CHECK(OrderStatus IN ('Pending', 'Shipped', 'Delivered', 'Cancelled')) NOT NULL,
    DeliveryDate TEXT,
    TotalPrice INTEGER CHECK (TotalPrice >= 0),
    ShippingPrice INTEGER CHECK (ShippingPrice >= 0),
    FOREIGN KEY (UserID) REFERENCES User(UserID) ON DELETE CASCADE
);

-- OrderProduct Table (Bridge Table for OrderDetail & Product)
CREATE TABLE IF NOT EXISTS OrderProduct (
    OrderProductID INTEGER PRIMARY KEY AUTOINCREMENT,
    OrderID INTEGER NOT NULL,
    ProductID INTEGER NOT NULL,
    Quantity INTEGER CHECK (Quantity > 0),
    PriceAtPurchase REAL CHECK (PriceAtPurchase >= 0),
    FOREIGN KEY (OrderID) REFERENCES OrderDetail(OrderID) ON DELETE CASCADE,
    FOREIGN KEY (ProductID) REFERENCES Product(ProductID) ON DELETE CASCADE
);

-- Product Table
CREATE TABLE IF NOT EXISTS Product (
    ProductID INTEGER PRIMARY KEY AUTOINCREMENT,
    Name TEXT NOT NULL,
    Description TEXT,
    StockQuantity INTEGER CHECK (StockQuantity >= 0),
    Colour TEXT,
    Weight REAL CHECK (Weight >= 0),
    Price REAL CHECK (Price >= 0),
    CategoryID INTEGER NOT NULL,
    FOREIGN KEY (CategoryID) REFERENCES Category(CategoryID) ON DELETE CASCADE
);

-- Category Table
CREATE TABLE IF NOT EXISTS Category (
    CategoryID INTEGER PRIMARY KEY AUTOINCREMENT,
    CategoryName TEXT UNIQUE NOT NULL
);

-- Brand Table
CREATE TABLE IF NOT EXISTS Brand (
    BrandID INTEGER PRIMARY KEY AUTOINCREMENT,
    BrandName TEXT UNIQUE NOT NULL
);

-- Carrier Table
CREATE TABLE IF NOT EXISTS Carrier (
    CarrierID INTEGER PRIMARY KEY AUTOINCREMENT,
    CarrierName TEXT NOT NULL,
    CarrierPhone TEXT,
    CarrierEmail TEXT
);

-- Review Table
CREATE TABLE IF NOT EXISTS Review (
    ReviewID INTEGER PRIMARY KEY AUTOINCREMENT,
    UserID INTEGER NOT NULL,
    ProductID INTEGER NOT NULL,
    Rating INTEGER CHECK (Rating BETWEEN 1 AND 5),
    ReviewText TEXT,
    ReviewDate TEXT DEFAULT CURRENT_TIMESTAMP,
    ReviewImage TEXT, -- Storing as a URL instead of BLOB
    FOREIGN KEY (UserID) REFERENCES User(UserID) ON DELETE CASCADE,
    FOREIGN KEY (ProductID) REFERENCES Product(ProductID) ON DELETE CASCADE
);

-- ShoppingCart Table
CREATE TABLE IF NOT EXISTS ShoppingCart (
    CartID INTEGER PRIMARY KEY AUTOINCREMENT,
    UserID INTEGER NOT NULL,
    DateAdded TEXT DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (UserID) REFERENCES User(UserID) ON DELETE CASCADE
);

-- Wishlist Table
CREATE TABLE IF NOT EXISTS Wishlist (
    WishlistID INTEGER PRIMARY KEY AUTOINCREMENT,
    UserID INTEGER NOT NULL,
    DateAdded TEXT DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (UserID) REFERENCES User(UserID) ON DELETE CASCADE
);

-- Discount Table
CREATE TABLE IF NOT EXISTS Discount (
    DiscountID INTEGER PRIMARY KEY AUTOINCREMENT,
    DiscountName TEXT NOT NULL,
    DiscountPercent REAL CHECK (DiscountPercent BETWEEN 0 AND 100),
    ExpireDate TEXT,
    Code TEXT UNIQUE NOT NULL
);

"""

# Execute schema script
cursor.executescript(schema_script)
conn.commit()

print("Database and tables created successfully in SQLite!")

# Close the connection
cursor.close()
conn.close()

In [ ]:
!pip install faker tqdm
import sqlite3
import random
from faker import Faker
from tqdm import tqdm
from datetime import datetime, timedelta
import string  # Import the string module

# Set a fixed seed for reproducibility
SEED = 42
random.seed(SEED)
faker = Faker(["en_GB", "fr_FR", "de_DE", "it_IT", "es_ES"])
faker.seed_instance(SEED)

# Define member types here
member_types = ['Silver', 'Gold', 'Diamond']

# Connect to SQLite database
conn = sqlite3.connect("luxury_ecommerce.db")
cursor = conn.cursor()

### --- Generate User Table Data --- ###
def generate_password(length=15):
    characters = string.ascii_letters + string.digits + "!@#$%^&*()_+"
    return "".join(random.choice(characters) for _ in range(random.randint(8, length)))

users = []
for i in tqdm(range(500), desc="Generating Users"):
    random.seed(i)  # Ensure consistent data per user
    first_name = faker.first_name()
    last_name = faker.last_name()
    email = f"{first_name.lower()}.{last_name.lower()}@{faker.domain_name()}"
    password = generate_password()
    phone_number = faker.phone_number()
    member_type = random.choice(member_types)
    users.append((email, password, first_name, last_name, member_type, phone_number))

cursor.executemany("INSERT INTO User (Email, Password, FirstName, LastName, MemberType, PhoneNumber) VALUES (?, ?, ?, ?, ?, ?)", users)

### --- Generate Category Table Data --- ###
categories = [
    (1, "Dresses & Gowns"), (2, "Suits & Blazers"), (3, "Tops & Shirts"),
    (4, "Jeans & Trousers"), (5, "Coats & Jackets"), (6, "Knitwear & Sweaters"),
    (7, "Skirts & Shorts"), (8, "Activewear"), (9, "Sneakers & Casual Shoes"),
    (10, "Accessories")
]
cursor.executemany("INSERT INTO Category (CategoryID, CategoryName) VALUES (?, ?)", categories)

### --- Generate Product Table Data --- ###
luxury_brands = ["Gucci", "Prada", "Louis Vuitton", "Hermès", "Dior", "Chanel", "Rolex", "Versace", "Armani", "Balenciaga"]
luxury_colours = ["Black", "White", "Gold", "Silver", "Navy Blue", "Emerald Green", "Burgundy", "Champagne", "Ivory", "Midnight Blue"]

# Define price ranges for different categories
category_price_ranges = {
    1: (500, 10000), 2: (1000, 15000), 3: (500, 5000),
    4: (700, 7000), 5: (2000, 20000), 6: (500, 5000),
    7: (500, 5000), 8: (800, 7000), 9: (1000, 12000), 10: (2000, 250000)
}

products = []
for i in range(100):
    random.seed(i)  # Ensure consistent product details
    category_id = categories[i % len(categories)][0]
    brand = random.choice(luxury_brands)
    product_name = f"{brand} {faker.word().capitalize()}"
    description = faker.sentence()
    min_price, max_price = category_price_ranges[category_id]
    price = random.randint(min_price, max_price)
    stock_quantity = random.randint(2, 50)
    colour = random.choice(luxury_colours)
    weight = round(random.uniform(0.5, 5.0), 2)
    products.append((product_name, description, stock_quantity, colour, weight, price, category_id))

cursor.executemany("INSERT INTO Product (Name, Description, StockQuantity, Colour, Weight, Price, CategoryID) VALUES (?, ?, ?, ?, ?, ?, ?)", products)

### --- Generate OrderDetail & OrderProduct Data --- ###
cursor.execute("SELECT ProductID, Price, StockQuantity FROM Product;")
product_data = cursor.fetchall()
cursor.execute("SELECT UserID FROM User;")
user_ids = [row[0] for row in cursor.fetchall()]
random.shuffle(user_ids)  # Shuffle users consistently

# Define the date range for orders (Q4 2024)
start_date = datetime(2024, 10, 1)
end_date = datetime(2024, 12, 31)
order_details = []
order_products = []

for order_id, user_id in enumerate(user_ids * 2, start=1):  # Ensure 1000 records
    random.seed(order_id)  # Ensure same products per order
    num_products = random.randint(1, 5)
    selected_products = random.sample(product_data, num_products)

    total_price = 0
    for product_id, price, stock in selected_products:
        quantity = random.randint(1, min(5, stock))
        total_price += price * quantity
        order_products.append((order_id, product_id, quantity, price))

    order_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
    order_status = random.choice(["Pending", "Shipped", "Delivered", "Cancelled"])
    delivery_date = order_date + timedelta(days=random.randint(2, 14)) if order_status in ["Shipped", "Delivered"] else None

    if total_price < 5000:
        shipping_price = random.randint(10, 50)
    elif total_price < 20000:
        shipping_price = random.randint(50, 100)
    else:
        shipping_price = random.randint(100, 200)

    if order_status in ["Pending", "Cancelled"]:
        shipping_price = 0

    order_details.append((user_id, order_date.strftime('%Y-%m-%d'), order_status, delivery_date.strftime('%Y-%m-%d') if delivery_date else None, total_price, shipping_price))

cursor.executemany("INSERT INTO OrderDetail (UserID, OrderDate, OrderStatus, DeliveryDate, TotalPrice, ShippingPrice) VALUES (?, ?, ?, ?, ?, ?)", order_details)
cursor.executemany("INSERT INTO OrderProduct (OrderID, ProductID, Quantity, PriceAtPurchase) VALUES (?, ?, ?, ?)", order_products)

conn.commit()
cursor.close()
conn.close()

print("All data successfully inserted into SQLite with consistent values.")


In [ ]:
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect("luxury_ecommerce.db")

# Define a function to query and display data from a specific table
def fetch_table_data(table_name, limit=500):
    """
    Fetch and display data from a given SQLite table.

    Parameters:
    table_name (str): The name of the table to query.
    limit (int): The number of rows to retrieve.
    """
    query = f"SELECT * FROM {table_name} LIMIT {limit};"
    df = pd.read_sql(query, conn)

    print(f"\n Displaying {limit} records from `{table_name}` table:")
    display(df)  # This will render the Pandas DataFrame properly in Colab

# Fetch and display data from multiple tables
fetch_table_data("User", limit=5)
fetch_table_data("OrderDetail", limit=5)
fetch_table_data("OrderProduct", limit=5)
fetch_table_data("Product", limit=5)
fetch_table_data("Category", limit=5)

# Close the database connection
conn.close()

In [ ]:
import sqlite3
import pandas as pd
from google.colab import files

# Connect to SQLite database
conn = sqlite3.connect("luxury_ecommerce.db")
cursor = conn.cursor()

# Query temp table for insights
# ---INSIGHT 1
# Identify customers by membertype

cursor.execute("""
              SELECT MEMBERTYPE
                , AVG(TTAL_PROD_COUNT) AVG_PROD_COUNT
                , AVG(AVG_PRODUCT_PURCHASE) AVG_PRODUCT_PURCHASE
                , SUM(TOTALPRICE) TOTALPRICE
                , AVG(TOTALPRICE) AVG_PURCHASE_PRICE
                , COUNT(DISTINCT USERID) CUSTOMERS
              FROM (
              SELECT A.ORDERID, A.USERID, A.ORDERDATE, A.ORDERSTATUS, A.DELIVERYDATE
                , CASE WHEN ORDERSTATUS NOT IN ('Delivered', 'Shipped') THEN NULL ELSE A.TOTALPRICE END AS TOTALPRICE
                , B.MEMBERTYPE
                , SUM(CASE WHEN ORDERSTATUS NOT IN ('Delivered', 'Shipped') THEN NULL ELSE C.QUANTITY END) TTAL_PROD_COUNT
                , AVG(CASE WHEN ORDERSTATUS NOT IN ('Delivered', 'Shipped') THEN NULL ELSE C.PRICEATPURCHASE END) AVG_PRODUCT_PURCHASE
                , SUM(CASE WHEN ORDERSTATUS NOT IN ('Delivered', 'Shipped') THEN NULL ELSE TOTALPRICE END) PURCHASE_PRICE_SUM

              FROM ORDERDETAIL A
              LEFT JOIN USER B ON A.USERID=B.USERID
              LEFT JOIN ORDERPRODUCT C ON A.ORDERID=C.ORDERID
              --WHERE --A.USERID='2' AND
              --ORDERSTATUS IN ('Delivered', 'Shipped')
              GROUP BY A.ORDERID, A.USERID, A.ORDERDATE, A.ORDERSTATUS, A.DELIVERYDATE, A.TOTALPRICE,
                     B.MEMBERTYPE
                     )A
              GROUP BY MEMBERTYPE
              ;
              """)

output = cursor.fetchall()
#print(output)
P_LOYALS = pd.DataFrame(output, columns=[desc[0] for desc in cursor.description])
print(P_LOYALS.to_string(index=False))

import matplotlib.pyplot as plt
import seaborn as sns


sns.set_style("whitegrid")
plt.rcParams["font.family"] = "DejaVu Sans"
gold = "#FFD700"
silver = "#C0C0C0"
diamond = "#B9F2FF"
custom_palette = [diamond, gold, silver]

plt.figure(figsize=(3, 3))
colors = sns.color_palette(custom_palette, len(P_LOYALS))

plt.pie(P_LOYALS["TOTALPRICE"], labels=P_LOYALS["MEMBERTYPE"],
        autopct="%1.1f%%", startangle=140, colors=colors,
        wedgeprops={'edgecolor': 'black', 'linewidth': 1.2})
plt.title("Sales Distribution by Membership Type", fontsize=16, pad=15)
plt.savefig("sales_membertype.png", dpi=300, bbox_inches="tight")
plt.show()
files.download("sales_membertype.png")



sns.set_style("whitegrid")
plt.rcParams["font.family"] = "DejaVu Sans"

plt.figure(figsize=(3, 3))
colors = sns.color_palette(custom_palette, len(P_LOYALS))

plt.pie(P_LOYALS["CUSTOMERS"], labels=P_LOYALS["MEMBERTYPE"],
        autopct="%1.1f%%", startangle=140, colors=colors,
        wedgeprops={'edgecolor': 'black', 'linewidth': 1.2})
plt.title("Customer Distribution by Membership Type", fontsize=16, pad=15)
plt.savefig("cust_membertype.png",dpi=300,bbox_inches="tight")
plt.show()
files.download("cust_membertype.png")




In [ ]:
# ---INSIGHT 2
# A. spending in each categories by royal/non royal customers
cursor.execute("""
               WITH AVG_MEMBERTYPE AS (
                  SELECT B.MEMBERTYPE
                    , AVG(A.TOTALPRICE) AS AVG_PURCHASE_PRICE
                 FROM ORDERDETAIL A
                 LEFT JOIN USER B ON A.USERID = B.USERID
                 WHERE ORDERSTATUS IN ('Delivered', 'Shipped')
                 GROUP BY B.MEMBERTYPE
              ),
              CUSTOMERS AS (
                  SELECT A.USERID
                    , B.MEMBERTYPE
                    , A.TOTALPRICE
                    , E.CATEGORYNAME
                    , SUM(C.QUANTITY*C.PRICEATPURCHASE) AS PURCHASEXCAT
                  FROM ORDERDETAIL A
                  LEFT JOIN USER B ON A.USERID = B.USERID
                  LEFT JOIN ORDERPRODUCT C ON A.ORDERID=C.ORDERID
                  LEFT JOIN PRODUCT D ON C.PRODUCTID=D.PRODUCTID
                  LEFT JOIN CATEGORY E ON D.CATEGORYID=E.CATEGORYID
                  WHERE ORDERSTATUS IN ('Delivered', 'Shipped')
                  GROUP BY A.USERID, B.MEMBERTYPE, A.TOTALPRICE, E.CATEGORYNAME
              )

             SELECT CATEGORYNAME
                --, COUNT(DISTINCT USERID) AS CUSTOMERCOUNT
                  , SUM(PURCHASEXCAT)PURCHASEXCAT, ROYAL
             FROM (
              SELECT A.USERID
                , A.MEMBERTYPE
                , A.TOTALPRICE
                , A.PURCHASEXCAT
                , A.CATEGORYNAME
                , CASE WHEN A.TOTALPRICE > B.AVG_PURCHASE_PRICE THEN 1 ELSE 0 END AS ROYAL
              FROM CUSTOMERS A
              LEFT JOIN AVG_MEMBERTYPE B ON A.MEMBERTYPE = B.MEMBERTYPE
             ) A
             GROUP BY  CATEGORYNAME, ROYAL

             ;
""")
output = cursor.fetchall()
#print(output)
df_royal_customers_a = pd.DataFrame(output, columns=[desc[0] for desc in cursor.description])
print(df_royal_customers_a.to_string(index=False))


# B. spending in each categories by membertype
cursor.execute("""
               WITH AVG_MEMBERTYPE AS (
                  SELECT B.MEMBERTYPE
                    , AVG(A.TOTALPRICE) AS AVG_PURCHASE_PRICE
                 FROM ORDERDETAIL A
                 LEFT JOIN USER B ON A.USERID = B.USERID
                 WHERE ORDERSTATUS IN ('Delivered', 'Shipped')
                 GROUP BY B.MEMBERTYPE
              ),
              CUSTOMERS AS (
                  SELECT A.USERID
                    , B.MEMBERTYPE
                    , A.TOTALPRICE
                    , E.CATEGORYNAME
                    , SUM(C.QUANTITY*C.PRICEATPURCHASE) AS PURCHASEXCAT
                  FROM ORDERDETAIL A
                  LEFT JOIN USER B ON A.USERID = B.USERID
                  LEFT JOIN ORDERPRODUCT C ON A.ORDERID=C.ORDERID
                  LEFT JOIN PRODUCT D ON C.PRODUCTID=D.PRODUCTID
                  LEFT JOIN CATEGORY E ON D.CATEGORYID=E.CATEGORYID
                  WHERE ORDERSTATUS IN ('Delivered', 'Shipped')
                  GROUP BY A.USERID, B.MEMBERTYPE, A.TOTALPRICE, E.CATEGORYNAME
              )

             SELECT CATEGORYNAME
                --, COUNT(DISTINCT USERID) AS CUSTOMERCOUNT
                  , SUM(PURCHASEXCAT)PURCHASEXCAT, MEMBERTYPE
             FROM (
              SELECT A.USERID
                , A.MEMBERTYPE
                , A.TOTALPRICE
                , A.PURCHASEXCAT
                , A.CATEGORYNAME
                , CASE WHEN A.TOTALPRICE > B.AVG_PURCHASE_PRICE THEN 1 ELSE 0 END AS ROYAL
              FROM CUSTOMERS A
              LEFT JOIN AVG_MEMBERTYPE B ON A.MEMBERTYPE = B.MEMBERTYPE
             ) A
             GROUP BY  CATEGORYNAME, MEMBERTYPE

             ;
""")
output = cursor.fetchall()
#print(output)
df_royal_customers_b = pd.DataFrame(output, columns=[desc[0] for desc in cursor.description])
print(df_royal_customers_b.to_string(index=False))

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


df_pivot = df_royal_customers_a.pivot(index="ROYAL", columns="CATEGORYNAME", values="PURCHASEXCAT").fillna(0)

df_pivot.index = df_pivot.index.map({0: "Non-Royal", 1: "Royal"})

df_pivot_percentage = df_pivot.div(df_pivot.sum(axis=1), axis=0) * 100

sns.set_style("whitegrid")
plt.rcParams["font.family"] = "DejaVu Sans"

fig, ax = plt.subplots(figsize=(10, 6))
df_pivot_percentage.plot(kind="bar", stacked=True, colormap="viridis", ax=ax)

plt.title("Spending Distribution by Royal vs. Non-Royal Customers & Category", fontsize=18, fontweight="bold", pad=15)
plt.xlabel("Customer Type", fontsize=14, fontweight="bold")
plt.ylabel("Percentage of Spending (%)", fontsize=14, fontweight="bold")
plt.xticks(rotation=0, fontsize=13)
plt.yticks(fontsize=13)
plt.legend(title="Category", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.grid(axis="y", linestyle="--", alpha=0.5)

for i, bars in enumerate(ax.containers):
    for bar in bars:
        height = bar.get_height()
        if height > 5:
            ax.text(
                bar.get_x() + bar.get_width() / 2,
                bar.get_y() + height / 2,
                f"{height:.1f}%",
                ha="center", va="center", fontsize=12, fontweight="bold", color="white"
            )
plt.savefig("royal_non-royal_stacked.png",dpi=300,bbox_inches="tight", transparent=True)
plt.show()
files.download("royal_non-royal_stacked.png")





In [ ]:
# ---INSIGHT 3
# A. sales per month by membertype
cursor.execute("""
              SELECT STRFTIME('%Y-%m', A.ORDERDATE) AS YEARMONTH
                , B.MEMBERTYPE
                , SUM(A.TOTALPRICE) TOTALPRICE
              FROM ORDERDETAIL A
              LEFT JOIN USER B ON A.USERID = B.USERID
              WHERE ORDERSTATUS IN ('Delivered', 'Shipped')
              GROUP BY  STRFTIME('%Y-%m', A.ORDERDATE) , B.MEMBERTYPE

             ;
""")
output = cursor.fetchall()
#print(output)
df_sales_a = pd.DataFrame(output, columns=[desc[0] for desc in cursor.description])
print(df_sales_a.to_string(index=False))

# B. sales per month by category
cursor.execute("""
              SELECT STRFTIME('%Y-%m', A.ORDERDATE) AS YEARMONTH, SUM(TOTALPRICE) TOTAL_PRICE, CATEGORYNAME
              FROM (
               SELECT A.USERID
                    , A.ORDERDATE
                    , A.TOTALPRICE
                    , E.CATEGORYNAME
               FROM ORDERDETAIL A
               LEFT JOIN ORDERPRODUCT C ON A.ORDERID=C.ORDERID
               LEFT JOIN PRODUCT D ON C.PRODUCTID=D.PRODUCTID
               LEFT JOIN CATEGORY E ON D.CATEGORYID=E.CATEGORYID
                  WHERE ORDERSTATUS IN ('Delivered', 'Shipped')
                  GROUP BY A.USERID, A.ORDERDATE, A.TOTALPRICE, E.CATEGORYNAME
                  ) A
              GROUP BY STRFTIME('%Y-%m', A.ORDERDATE), CATEGORYNAME
             ;
""")
output = cursor.fetchall()
#print(output)
df_sales_b = pd.DataFrame(output, columns=[desc[0] for desc in cursor.description])
print(df_sales_b.to_string(index=False))

# C1. sales per month by status -- nice to se % of sales ($) cancelled or pending
cursor.execute("""
               SELECT STRFTIME('%Y-%m', A.ORDERDATE) AS YEARMONTH
                , CASE WHEN A.ORDERSTATUS ='Cancelled' THEN '4.Cancelled'
                       WHEN A.ORDERSTATUS ='Pending' THEN '3.Pending'
                       WHEN A.ORDERSTATUS ='Shipped' THEN '2.Shipped'
                       WHEN A.ORDERSTATUS ='Delivered' THEN '1.Delivered'
                       END AS OrderStatus
                , SUM(A.TOTALPRICE) TOTALPRICE
                , COUNT(A.ORDERID) ORDER_COUNT
              FROM ORDERDETAIL A
              GROUP BY  STRFTIME('%Y-%m', A.ORDERDATE) , CASE WHEN A.ORDERSTATUS ='Cancelled' THEN '4.Cancelled'
                       WHEN A.ORDERSTATUS ='Pending' THEN '3.Pending'
                       WHEN A.ORDERSTATUS ='Shipped' THEN '2.Shipped'
                       WHEN A.ORDERSTATUS ='Delivered' THEN '1.Delivered'
                       END
             ;
""")
output = cursor.fetchall()
#print(output)
df_sales_c1 = pd.DataFrame(output, columns=[desc[0] for desc in cursor.description])
print(df_sales_c1.to_string(index=False))

# C2. sales per month by status -- nice to se % of sales(# of orders) cancelled or pending
cursor.execute("""
               SELECT STRFTIME('%Y-%m', A.ORDERDATE) AS YEARMONTH
                , A.ORDERSTATUS
                , SUM(A.TOTALPRICE) TOTALPRICE
                , COUNT(A.ORDERID) ORDER_COUNT
              FROM ORDERDETAIL A
              GROUP BY  STRFTIME('%Y-%m', A.ORDERDATE) , A.ORDERSTATUS
             ;
""")
output = cursor.fetchall()
#print(output)
df_sales_c2 = pd.DataFrame(output, columns=[desc[0] for desc in cursor.description])
print(df_sales_c2.to_string(index=False))

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

df_sales_a["YEARMONTH"] = pd.to_datetime(df_sales_a["YEARMONTH"], format="%Y-%m")

df_sales_a["TOTALPRICE"] = df_sales_a["TOTALPRICE"].astype(float)

df_full = df_sales_a.pivot(index="YEARMONTH", columns="MemberType", values="TOTALPRICE").reset_index()

sns.set_style("whitegrid")
plt.rcParams["font.family"] = "DejaVu Sans"
plt.figure(figsize=(14, 7))

sns.lineplot(data=df_full, x="YEARMONTH", y=df_full["Diamond"]/1e6, marker="o", linewidth=2, label="Diamond", color="#B9F2FF")
sns.lineplot(data=df_full, x="YEARMONTH", y=df_full["Gold"]/1e6, marker="o", linewidth=2, label="Gold", color="#FFD700")
sns.lineplot(data=df_full, x="YEARMONTH", y=df_full["Silver"]/1e6, marker="o", linewidth=2, label="Silver", color="#C0C0C0")
plt.title("Monthly Sales by Membership Type", fontsize=20, fontweight="bold", pad=15)
plt.xlabel("Month", fontsize=16, fontweight="bold")
plt.ylabel("Total Sales in Millions($)", fontsize=16, fontweight="bold")
plt.xticks(df_full["YEARMONTH"], df_full["YEARMONTH"].dt.strftime("%b %Y"), rotation=45, fontsize=14)
plt.ylim(0, (df_full[["Diamond", "Gold", "Silver"]].max().max() / 1e6) * 1.1)
plt.yticks(fontsize=14)
plt.legend(title="Membership Type", fontsize=14, title_fontsize=16, bbox_to_anchor=(1.05, 1), loc="upper left")
plt.grid(axis="y", linestyle="--", alpha=0.6)
plt.ticklabel_format(style='plain', axis='y')
plt.savefig("monthly_sales_membtype.png",dpi=300,bbox_inches="tight")
plt.show()
files.download("monthly_sales_membtype.png")



df_sales_c1["YEARMONTH"] = pd.to_datetime(df_sales_c1["YEARMONTH"], format="%Y-%m")
df_sales_c1["TOTALPRICE"] = df_sales_c1["TOTALPRICE"].astype(float)
df_pivot = df_sales_c1.pivot(index="YEARMONTH", columns="OrderStatus", values="TOTALPRICE").fillna(0)
df_pivot_percentage = df_pivot.div(df_pivot.sum(axis=1), axis=0) * 100  # Convert to percentages

sns.set_style("whitegrid")
plt.rcParams["font.family"] = "DejaVu Sans"
status_colors = {
    "1.Delivered": "#1B9E77",
    "2.Shipped": "#D95F02",
    "3.Pending": "#7570B3",
    "4.Cancelled": "#E7298A"
}

fig, ax = plt.subplots(figsize=(14, 7))
df_pivot_percentage.plot(kind="bar", stacked=True, ax=ax, color=[status_colors[col] for col in df_pivot_percentage.columns])

plt.title("Percentage of Sales by Order Status Per Month", fontsize=20, fontweight="bold", pad=15)
plt.xlabel("Month", fontsize=16, fontweight="bold")
plt.ylabel("Percentage of Total Sales (%)", fontsize=16, fontweight="bold")
plt.xticks(ticks=range(len(df_pivot_percentage.index)), labels=df_pivot_percentage.index.strftime("%b %Y"), rotation=45, fontsize=14)
plt.yticks(fontsize=14)
plt.legend(title="Order Status", fontsize=14, title_fontsize=16, bbox_to_anchor=(1.05, 1), loc="upper left")
plt.grid(axis="y", linestyle="--", alpha=0.6)
for i, rect in enumerate(ax.patches):
    height = rect.get_height()
    if height > 5:
        ax.text(
            rect.get_x() + rect.get_width() / 2,
            rect.get_y() + height / 2,
            f"{height:.1f}%",
            ha="center", va="center", fontsize=12, fontweight="bold", color="white"
        )
plt.savefig("monthly_sales_order-status.png",dpi=300,bbox_inches="tight")
plt.show()
files.download("monthly_sales_order-status.png")








In [ ]:
cursor.execute("""

          SELECT PRODUCTNAME, SUM(QUANTITY)/3 AS QUANTITY_MONTH, STOCKQUANTITY, STOCKQUANTITY*PRICE VALUE_INVENTORY
              FROM (
              SELECT A.USERID
              , A.ORDERDATE
              , C.QUANTITY
              , C.PRODUCTID
              , D.NAME AS PRODUCTNAME
              , D.STOCKQUANTITY
              , D.PRICE
              FROM ORDERDETAIL A
              LEFT JOIN ORDERPRODUCT C ON A.ORDERID=C.ORDERID
              LEFT JOIN PRODUCT D ON C.PRODUCTID=D.PRODUCTID
              WHERE ORDERSTATUS IN ('Delivered', 'Shipped')
             -- AND USERID=2
              -- GROUP BY A.USERID, B.MEMBERTYPE, A.TOTALPRICE, E.CATEGORYNAME
              ) A
              GROUP BY PRODUCTNAME, STOCKQUANTITY, STOCKQUANTITY*PRICE
              ORDER BY QUANTITY_MONTH ASC
              LIMIT 10
              ;
              """)
output = cursor.fetchall()
#print(output)
df_best_sellers = pd.DataFrame(output, columns=[desc[0] for desc in cursor.description])
print(df_best_sellers.to_string(index=False))

sns.set_style("whitegrid")
plt.rcParams["font.family"] = "DejaVu Sans"

fig, ax1 = plt.subplots(figsize=(12, 6))

sns.barplot(y="PRODUCTNAME", x="QUANTITY_MONTH", data=df_best_sellers,
            palette="magma", edgecolor="black", ax=ax1)

for index, row in df_best_sellers.iterrows():
    ax1.text(row["QUANTITY_MONTH"] + 0.3, index, f"{row['QUANTITY_MONTH']:.1f}",
             va="center", fontsize=12, fontweight="bold", color="black")

ax1.set_title("Top 10 Least-Selling Products (Last Quarter)", fontsize=16, fontweight="bold", pad=15)
ax1.set_xlabel("Average Monthly Quantity Sold", fontsize=14, fontweight="bold")
ax1.set_ylabel("Product Name", fontsize=14, fontweight="bold")
ax1.grid(axis="x", linestyle="--", alpha=0.5)
plt.savefig("least_selling_products.png",dpi=300,bbox_inches="tight")
plt.show()
files.download("least_selling_products.png")

In [ ]:
# ---INSIGHT 4
# Top 10 Best Selling Products, avg quantity sold in the last quarter, remaining inventory
cursor.execute("""

              SELECT PRODUCTNAME
                , SUM(QUANTITY)/3 AS QUANTITY_MONTH
                , STOCKQUANTITY
                , STOCKQUANTITY / (SUM(QUANTITY)/3)
                , ROUND((STOCKQUANTITY / (SUM(QUANTITY)/3)) * 100, 2)  AS INV_AVAILABLE_1M
              FROM (
              SELECT A.USERID
              , A.ORDERDATE
              , C.QUANTITY
              , C.PRODUCTID
              , D.NAME AS PRODUCTNAME
              , D.STOCKQUANTITY
              , D.PRICE
              FROM ORDERDETAIL A
              LEFT JOIN ORDERPRODUCT C ON A.ORDERID=C.ORDERID
              LEFT JOIN PRODUCT D ON C.PRODUCTID=D.PRODUCTID
              WHERE ORDERSTATUS IN ('Delivered', 'Shipped')
             -- AND USERID=2
              -- GROUP BY A.USERID, B.MEMBERTYPE, A.TOTALPRICE, E.CATEGORYNAME
              ) A
              GROUP BY PRODUCTNAME, STOCKQUANTITY
              ORDER BY QUANTITY_MONTH DESC
              LIMIT 10
              ;
              """)
output = cursor.fetchall()
#print(output)
# Check for small values and avoid 0% results
df_best_sellers = pd.DataFrame(output, columns=[desc[0] for desc in cursor.description])

# If INV_AVAILABLE_1M_PERCENTAGE is too small (close to 0 or 100), it could indicate an imbalance in stock vs quantity sold.
df_best_sellers['INV_AVAILABLE_1M'] = df_best_sellers['INV_AVAILABLE_1M'].apply(lambda x: min(max(x, 0), 100))  # Clip values between 0 and 100

print(df_best_sellers.to_string(index=False))

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")
plt.rcParams["font.family"] = "DejaVu Sans"

fig, ax1 = plt.subplots(figsize=(12, 6))

sns.barplot(y="PRODUCTNAME", x="QUANTITY_MONTH", data=df_best_sellers,
            palette="magma", edgecolor="black", ax=ax1)

for index, row in df_best_sellers.iterrows():
    ax1.text(row["QUANTITY_MONTH"] + 1, index, f"{row['QUANTITY_MONTH']:.1f}",
             va="center", fontsize=12, fontweight="bold", color="black")

ax1.set_title("Top 10 Best-Selling Products (Last Quarter)", fontsize=16, fontweight="bold", pad=15)
ax1.set_xlabel("Average Monthly Quantity Sold", fontsize=14, fontweight="bold")
ax1.set_ylabel("Product Name", fontsize=14, fontweight="bold")
ax1.grid(axis="x", linestyle="--", alpha=0.5)
plt.savefig("best_selling_products.png",dpi=300,bbox_inches="tight")
plt.show()
files.download("best_selling_products.png")


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# ---INSIGHT 4
# Top 10 Best Selling Products, avg quantity sold in the last quarter, remaining inventory
cursor.execute("""
              SELECT A.USERID
              , C.QUANTITY
              , C.PRICEATPURCHASE
              , C.PRODUCTID
              , D.NAME AS PRODUCTNAME
              , D.STOCKQUANTITY
              FROM ORDERDETAIL A
              LEFT JOIN ORDERPRODUCT C ON A.ORDERID=C.ORDERID
              LEFT JOIN PRODUCT D ON C.PRODUCTID=D.PRODUCTID
              WHERE ORDERSTATUS IN ('Delivered', 'Shipped')
              AND USERID=2
              -- GROUP BY A.USERID, B.MEMBERTYPE, A.TOTALPRICE, E.CATEGORYNAME
              ;
              """)
output = cursor.fetchall()
#print(output)
df_best_sellers = pd.DataFrame(output, columns=[desc[0] for desc in cursor.description])
print(df_best_sellers.to_string(index=False))
sns.set_style("whitegrid")
plt.rcParams["font.family"] = "Dejavu Sans"

plt.figure(figsize=(12, 6))
ax = sns.barplot(y="PRODUCTNAME", x="TotalQuantitySold",hue="PRODUCTNAME", data=df_best_sellers,
                 palette="rocket", edgecolor="black",legend=False)

for index, row in df_best_sellers.iterrows():
    ax.text(row["TotalQuantitySold"] + 1, index, f"{row['TotalQuantitySold']:,}",
            va="center", fontsize=12, color="black")

plt.title("Top 10 Best-Selling Products", fontsize=18, pad=15)
plt.xlabel("Total Quantity Sold", fontsize=14)
plt.ylabel("Product Name", fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(axis="x", linestyle="--", alpha=0.5)
sns.despine(left=True, bottom=True)

plt.show()

In [ ]:
#Top 10 Least Selling Products
cursor.execute("""SELECT PRODUCTID, PRODUCTNAME, SUM(QUANTITY) AS TotalQuantitySold
FROM TempTable
WHERE ORDERSTATUS = 'Delivered'
GROUP BY PRODUCTID, PRODUCTNAME
ORDER BY TotalQuantitySold ASC
LIMIT 10;
""")
output = cursor.fetchall()
#print(output)
df_least_sellers = pd.DataFrame(output, columns=[desc[0] for desc in cursor.description])
print(df_least_sellers.to_string(index=False))
sns.set_style("whitegrid")
plt.rcParams["font.family"] = "Dejavu Sans"

plt.figure(figsize=(12, 6))
ax = sns.barplot(y="PRODUCTNAME", x="TotalQuantitySold", data=df_least_sellers,
                 palette="magma", edgecolor="black")

for index, row in df_least_sellers.iterrows():
    ax.text(row["TotalQuantitySold"] + 0.4, index, f"{row['TotalQuantitySold']:,}",
            va="center", fontsize=12, color="black")

plt.title("Top 10 Least-Selling Products", fontsize=18, pad=15)
plt.xlabel("Total Quantity Sold", fontsize=14)
plt.ylabel("Product Name", fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(axis="x", linestyle="--", alpha=0.5)
sns.despine(left=True, bottom=True)

plt.show()

In [ ]:
#Top 3 Best Selling Categories
cursor.execute("""SELECT CATEGORYNAME, SUM(QUANTITY) AS TotalQuantitySold
FROM TempTable
WHERE ORDERSTATUS = 'Delivered'
GROUP BY CATEGORYNAME
ORDER BY TotalQuantitySold DESC
LIMIT 3;
""")
output = cursor.fetchall()
#print(output)
df_best_categories = pd.DataFrame(output, columns=[desc[0] for desc in cursor.description])
print(df_best_categories.to_string(index=False))

In [ ]:
#Top 3 Highest Revenue generating category
cursor.execute("""SELECT CATEGORYNAME, SUM(TOTALPRICE) AS TotalRevenue
FROM TempTable
WHERE ORDERSTATUS = 'Delivered'
GROUP BY CATEGORYNAME
ORDER BY TotalRevenue DESC
LIMIT 3;""")
output = cursor.fetchall()
#print(output)
df_highest_revenue_category = pd.DataFrame(output, columns=[desc[0] for desc in cursor.description])
print(df_highest_revenue_category.to_string(index=False))

In [ ]:
#Total Amount Spent by different Membership Types in Q4
import matplotlib.pyplot as plt
import seaborn as sns
cursor.execute("""SELECT MEMBERTYPE, SUM(TOTALPRICE) AS TotalPurchaseAmount
FROM TempTable
WHERE ORDERSTATUS = 'Delivered'
GROUP BY MEMBERTYPE
ORDER BY TotalPurchaseAmount DESC;
""")
output = cursor.fetchall()
#print(output)
df_membertype = pd.DataFrame(output, columns=[desc[0] for desc in cursor.description])
print(df_membertype.to_string(index=False))
sns.set_style("whitegrid")
plt.rcParams["font.family"] = "Dejavu Sans"

plt.figure(figsize=(12, 6))
ax = sns.barplot(x="MemberType", y="TotalPurchaseAmount",hue="TotalPurchaseAmount", data=df_membertype, palette="crest", edgecolor="black",legend=False)

for index, row in df_membertype.iterrows():
    ax.text(index, row["TotalPurchaseAmount"] + (row["TotalPurchaseAmount"] * 0.02),
            f"${row['TotalPurchaseAmount']:,.2f}",
            ha="center", fontsize=12, color="black")

plt.title(" Total Amount Spent by Membership Type ", fontsize=16, pad=15)
plt.xlabel("Membership Type", fontsize=13)
plt.ylabel("Total Amount Spent ($)", fontsize=13)
plt.xticks(rotation=0, fontsize=12)
plt.yticks(fontsize=12)
plt.grid(axis="y", linestyle="--", alpha=0.5)
sns.despine(left=True, bottom=True)

plt.show()